In [28]:
import langchain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import re
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.load import dumps, loads

In [2]:
load_dotenv()

In [3]:
doc_path = '../data/pdf.pdf'

In [21]:
def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [18]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template='You are a helpful assistant that generates multiple search queries based on a single input query.')),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (3 queries):'))])

In [4]:
def preprocess_text(text: str) -> str:

    text = text.lower()
    text = text.encode("ascii", errors="ignore").decode()
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\n', ' ').strip()
    return text

In [ ]:
loader=PyPDFLoader(doc_path)
docs=loader.load()
docs

In [7]:
for doc in docs:
    doc.page_content = preprocess_text(doc.page_content)

In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=30)
chunks = splitter.split_documents(docs)

In [9]:
embeddings = OpenAIEmbeddings()

In [10]:
vectorstore=Chroma.from_documents(chunks,embeddings)
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 5})
vectorstore_retreiver

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000015CF684C5B0>, search_kwargs={'k': 5})

In [11]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k = 3
keyword_retriever

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x0000015C917FDA60>, k=3)

In [12]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,keyword_retriever],weights=[0.3, 0.7])
ensemble_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000015CF684C5B0>, search_kwargs={'k': 5}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x0000015C917FDA60>, k=3)], weights=[0.3, 0.7])

In [22]:
llm = ChatOpenAI()

In [ ]:
generate_queries = (
    prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
)
generate_queries

In [29]:
ragfusion_chain = generate_queries | ensemble_retriever.map() | reciprocal_rank_fusion
langchain.debug = True
ragfusion_chain.input_schema.schema()

C:\Users\Hp\AppData\Local\Temp\ipykernel_7436\249600610.py:3: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  ragfusion_chain.input_schema.schema()


{'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question'],
 'title': 'PromptInput',
 'type': 'object'}

In [31]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

full_rag_fusion_chain = (
    {
        "context": ragfusion_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

full_rag_fusion_chain.input_schema.schema()

C:\Users\Hp\AppData\Local\Temp\ipykernel_7436\690582261.py:18: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  full_rag_fusion_chain.input_schema.schema()


{'properties': {'question': {'title': 'Question', 'type': 'string'},
  'root': {'title': 'Root'}},
 'required': ['question', 'root'],
 'title': 'RunnableParallel<context,question>Input',
 'type': 'object'}

In [38]:
response1 = full_rag_fusion_chain.invoke("How to prevent data contamination?")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "How to prevent data contamination?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "How to prevent data contamination?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "How to prevent data contamination?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "How to prevent data contamination?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > llm:ChatOpenAI] Ente

In [39]:
response1

'To prevent data contamination, it is important to understand what is in the pretraining data, remove data from certain sites known to contain a high volume of personal information about private individuals, and use a conservative approach to produce a clean subset of the data with high precision. It is also essential to conduct data checks even when using vendors to source annotations and employ processes to mitigate safety risks related to the data.'